## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

1. **What percentage of users have invalid passwords?** Save your answer as a variable, `bad_pass`, in the form of a float rounded up to two decimals (e.g., 0.18).

2. **Which users need to change their passwords?** Save your answer as a `pandas` Series consisting of the `usernames` in *alphabetically ascending order* called `email_list`. This will be used to automate email notifications to employees.

In [61]:
import pandas as pd, numpy as np, re, string

df = pd.read_csv('datasets/logins.csv')
df['first_name'] = df['username'].str.extract(r"(^[A-Za-z]+)", expand=False)
df['last_name'] = df['username'].str.extract(r"([A-Za-z]+$)", expand=False)

df['too_short'] = df['password'].str.len() < 10
df['contains_password'] = df['password'].str.lower().str.contains('password')

# อันนี้ ใช้ .str.contains ไม่ได้ จึงต้องใช้ iterrows แทน
# df['contains_name'] = (df['first_name'].str.lower().str.contains(df['password'].str.lower())) | (df['last_name'].str.lower().str.contains(df['password'].str.lower()))

df['contains_names'] = False
for i, j in df.iterrows():
    if (j['first_name'].lower() in j['password'].lower()) | (j['last_name'].lower() in j['password'].lower()):
        df.loc[i, 'contains_names'] = True

df['not_contain_non_alpha_numeric'] = True
for i, j in df.iterrows():
    for k in string.punctuation:
        if k in j['password']:
            df.loc[i, 'not_contain_non_alpha_numeric'] = False
            break
            
df['not_contain_lowercase'] = True
for i, j in df.iterrows():
    if re.search(r"[a-z]", j['password']):
        df.loc[i, 'not_contain_lowercase'] = False
        
df['not_contain_uppercase'] = True
for i, j in df.iterrows():
    if re.search(r"[A-Z]", j['password']):
        df.loc[i, 'not_contain_uppercase'] = False
        
df['not_contain_number'] = True
for i, j in df.iterrows():
    if re.search(r"[0-9]", j['password']):
        df.loc[i, 'not_contain_number'] = False

df['bad_password'] = df['too_short'] | df['contains_password'] | df['contains_names'] | df['not_contain_lowercase'] | df['not_contain_uppercase'] | df['not_contain_number'] | df['not_contain_non_alpha_numeric'] 
        
bad_pass = round(df[df['bad_password']].shape[0] / df.shape[0], 2)

In [57]:
df[df['bad_password']].sample(5)

,id,username,password,first_name,last_name,too_short,contains_password,contains_names,not_contain_non_alpha_numeric,not_contain_lowercase,not_contain_uppercase,not_contain_number,bad_password
167,168,christy.lindsey,"Y,O&jo_n",christy,lindsey,True,False,False,False,False,False,True,True
634,635,zachary.huff,78momma,zachary,huff,True,False,False,True,False,True,False,True
594,595,willis.cervantes,9812684,willis,cervantes,True,False,False,True,True,True,False,True
181,182,loretta.dickson,76414592916,loretta,dickson,False,False,False,True,True,True,False,True
169,170,greta.english,h28hactsG,greta,english,True,False,False,True,False,False,False,True


In [62]:
email_list = df[df['bad_password']]['username'].sort_values()
email_list

931           abdul.rowland
713            addie.cherry
857            adele.moreno
291            adeline.bush
663             adolfo.kane
775             adolfo.lara
51             ahmad.hopper
298              aida.combs
898           aisha.jenkins
471               al.dunlap
356            alana.franco
546         alberta.leblanc
306            alec.robbins
831    alejandra.stephenson
44          alejandro.burke
195        alejandro.nieves
483        alexander.thomas
920       alexandria.hinton
93        alexis.mccullough
219         alexis.reynolds
456          alfonso.weaver
366           alfonzo.johns
595          alisa.campbell
781             alisa.cohen
442             alison.neal
452          allan.marshall
338           alonzo.fowler
751           amado.bridges
207        amado.fitzgerald
543           amber.summers
               ...         
64              ursula.wood
664       valentin.castillo
551           valeria.curry
0            vance.jennings
731           vaness

In [60]:
# เฉลย คำตอบเหมือน แต่ไม่ให้ผ่าน งี่เง่าจริงๆ

# Importing the pandas module
import pandas as pd

# Loading in datasets/users.csv 
logins = pd.read_csv("datasets/logins.csv")

# Rule 1: Not too short
# Create a boolean variable
length_check = logins['password'].str.len() >= 10
# Separate using boolean indexing
valid_pws = logins[length_check]
bad_pws = logins[~length_check]

# Rule 2: All the types of characters
# Let's create a boolean index for each character requirement
# [ ] is used to indicate a set of characters
# e.g. [abc] will match 'a', 'b', or 'c'.
# We can use a-z to represent all lowercase chars between a and Z
lcase = valid_pws['password'].str.contains('[a-z]') 
ucase = valid_pws['password'].str.contains('[A-Z]')
special = valid_pws['password'].str.contains('\W')
# \d matches any decimal digit; this is equivalent to doing [0-9]
# \W matches any non-alphanumeric character
numeric = valid_pws['password'].str.contains('\d')
# A password needs to have all these as true 
# If any of these are false, we need it to return false
# In other words, all of these have to be true to return true
# We can use the & (and) operator
char_check = lcase & ucase & numeric & special
bad_pws = bad_pws.append(valid_pws[~char_check],ignore_index=True)
valid_pws = valid_pws[char_check]

# Rule 3: Must not contain the phrase password (case insensitive)
banned_phrases = valid_pws['password'].str.contains('password', case=False) 
bad_pws = bad_pws.append(valid_pws[banned_phrases],ignore_index=True)
valid_pws = valid_pws[~banned_phrases]

# Rule 4: Must not contain the user's first or last name
# Extracting first and last names into their own columns
valid_pws['first_name'] = valid_pws['username'].str.extract('(^\w+)', expand = False)
valid_pws['last_name'] = valid_pws['username'].str.extract('(\w+$)', expand = False)
# Iterate over DataFrame rows
for i, row in valid_pws.iterrows():
    if row.first_name in row.password.lower() or row.last_name in row.password.lower():
        valid_pws = valid_pws.drop(index=i)
        bad_pws = bad_pws.append(row,ignore_index=True)
# Note this could be done more efficiently with a lambda function

# Answering the questions
bad_pass = round(bad_pws.shape[0] / logins.shape[0], 2)
print("Percentage of users with invalid passwords", bad_pass)
email_list = bad_pws['username'].sort_values()
print(email_list)

Percentage of users with invalid passwords 0.75
405           abdul.rowland
309            addie.cherry
372            adele.moreno
517            adeline.bush
279             adolfo.kane
337             adolfo.lara
16             ahmad.hopper
122              aida.combs
700           aisha.jenkins
199               al.dunlap
147            alana.franco
593         alberta.leblanc
521            alec.robbins
671    alejandra.stephenson
434         alejandro.burke
482        alejandro.nieves
205        alexander.thomas
400       alexandria.hinton
453       alexis.mccullough
93          alexis.reynolds
568          alfonso.weaver
151           alfonzo.johns
611          alisa.campbell
342             alisa.cohen
567             alison.neal
190          allan.marshall
142           alonzo.fowler
652           amado.bridges
88         amado.fitzgerald
592           amber.summers
               ...         
20              ursula.wood
280       valentin.castillo
596           valeria.curry
